In [5]:
import ipywidgets as widgets
from IPython.display import display, HTML

javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Show code", True: "Hide code"}


def toggle_code(state):

    """
    Toggles the JavaScript show()/hide() function on the div.input element.
    """

    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)

    display(HTML(output))

def button_action(value):

    """
    Calls the toggle_code function and updates the button description.
    """

    state = value.new

    toggle_code(state)

    value.owner.description = button_descriptions[state]


state = False
toggle_code(state)

button = widgets.ToggleButton(state, description = button_descriptions[state])
button.observe(button_action, "value")

display(button)

import numpy as np
import ipyvolume as ipv
import ipywidgets as ipw
import scipy
from scipy.spatial.transform import Rotation as rot
from scipy.spatial.distance import cdist
import math
import matplotlib
import matplotlib.pyplot as plt
import random
from random import uniform

def cart2sph(v):
    x,y,z = v
    XsqPlusYsq = x**2 + y**2
    r = np.sqrt(XsqPlusYsq + z**2)                  # r
    elev = math.atan2(z,np.sqrt(XsqPlusYsq))        # theta
    az = math.atan2(y,x)                            # phi
    return np.array([r, elev, az])

def sph2cart(v):
    r, elev, az = v
    x = r * np.sin(elev) * np.cos(az)
    y = r * np.sin(elev) * np.sin(az)
    z = r * np.cos(elev)
    return np.array([x,y,z])

def angle(v1, v2):
    """ Returns the angle in degrees between vectors 'v1' and 'v2'"""
    v1_u = v1/np.linalg.norm(v1)
    v2_u = v2/np.linalg.norm(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))*180/np.pi

def random_point_on_circle(r):
    angle = np.random.uniform(0.,2.*np.pi)
    x = np.cos(angle)*r 
    y = np.sin(angle)*r
    return np.array([x,y])

def random_point_on_lattice(r):
    choices = np.array([[0,1],[0,-1],[1,0],[-1,0]])
    idx = np.random.choice(len(choices),1)
    return choices[idx]*r


def random_point_on_shpere(R):
    """ Returns points on a sphere with radius R, uniformly distributed"""
    radius = R*(np.random.uniform(0.,1.))**(1./3.)
    theta = np.arccos(np.random.uniform(-1.,1.))
    phi = np.random.uniform(0.,2.*np.pi)
    v = sph2cart([radius,theta,phi])
    return v

def regular_points_on_sphere(n,r):
    coords = []
    alpha = 4.0*np.pi*r*r/n
    d = np.sqrt(alpha)
    m_nu = int(np.round(np.pi/d))
    d_nu = np.pi/m_nu
    d_phi = alpha/d_nu
    count = 0
    for m in range (0,m_nu):
        nu = np.pi*(m+0.5)/m_nu
        m_phi = int(np.round(2*np.pi*np.sin(nu)/d_phi))
        for n in range (0,m_phi):
            phi = 2*np.pi*n/m_phi
            point = sph2cart([r,nu,phi])
            coords.append(point)
    return np.asarray(coords)

def random_point_on_cone(R,theta,prev):
    """ Returns random vector with length R and angle theta between previos one, uniformly distributed"""
    theta *=np.pi/180.
    v = prev/np.linalg.norm(prev)
    # find "mostly orthogonal" vector to prev
    a = np.zeros((3,))
    a[np.argmin(np.abs(prev))]=1
    #print(a,v,prev)
    # find orthonormal coordinate system {x_hat, y_hat, v}
    x_hat = np.cross(a,v)/np.linalg.norm(np.cross(a,v))
    y_hat = np.cross(v,x_hat)
    # draw random rotation 
    phi = np.random.uniform(0.,2.*np.pi) 
    # determine vector (random rotation + rotation with theta to guarantee the right angle between v,w)
    w = np.sin(theta)*np.cos(phi)*x_hat + np.sin(theta)*np.sin(phi)*y_hat + np.cos(theta)*v
    w *=R
    return w

def random_coords(N,model,theta):
    coords = np.zeros((int(N),3))
    coords[1]=[1,0,0]
    for i in range(2,int(N)):
        if model == 'freely_jointed':
            n = random_point_on_shpere(1.0)
            new = coords[i-1]+n
        if model == 'freely_rotating':
            prev = coords[i-2]-coords[i-1]
            n = random_point_on_cone(1.0,theta,prev)
            new = coords[i-1]+n
        coords[i]=new
    return coords


def random_coords_2D(N,type):
    coords = np.zeros((int(N),2))
    coords[1]=[1.,0.]
    for i in range(2,int(N)):
        if type=='continous':
            n = random_point_on_circle(1.0)
        elif type=='lattice':
            n = random_point_on_lattice(1.0)
        new = coords[i-1]+n
        coords[i]=new
    return coords

def random_walk(N):
    steps = np.random.randint(low = -1, high = 1, size = (N))*2 +1 
    random_trajectory = np.cumsum(steps)
    random_trajectory = np.concatenate([[0],random_trajectory])
    
    return random_trajectory

def calculate_overlaps(coords):
    d = cdist(coords,coords)
    i = np.arange(len(coords)).reshape(len(coords),1)
    u = cdist(i,i)
    q = np.where(np.logical_and(u > 1,d<0.5) , True, False)
    l = np.argwhere(q)
    l = np.unique(np.vstack((l[:,0],l[:,1])))
    return coords[l.astype(int)]

def random(n,type,draw_beads):
    a = random_coords_2D(n,type)
    ipv.figure(height=400,width=600)
    plt.plot(a[:,0],a[:,1],color='black')
    if draw_beads:
        plt.scatter(a[:,0],a[:,1],color='red')
    plt.show()
    return a

def random(n,draw_beads):
    a = random_coords(n,'freely_jointed',0)
    ipv.figure(height=400,width=600)
    ipv.pylab.plot(a[:,0],a[:,1],a[:,2],color='black')
    if draw_beads:
        ipv.pylab.scatter(a[:,0],a[:,1],a[:,2],color='red')
    ipv.pylab.xyzlabel(' ',' ',' ')
    
    max = np.max(a)
    min = np.min(a)
    ipv.pylab.xlim(min, max)
    ipv.pylab.ylim(min, max)
    ipv.pylab.zlim(min, max)
    ipv.show()
    return a

def f(model='freely_jointed',theta=0.0,n=10,draw_end_to_end=False,draw_radius_of_gyration=False,draw_overlaps=False):
    a = random_coords(n,model,180-theta)
    ipv.figure(height=400,width=600)
    if draw_end_to_end:
        end_to_end_dist = a[-1]-a[0]
        ipv.pylab.plot([0,end_to_end_dist[0]],[0,end_to_end_dist[1]],[0,end_to_end_dist[2]])
        ipv.pylab.scatter((a[0][0],a[-1][0]),(a[0][1],a[-1][1]),(a[0][2],a[-1][2]),color='red')

    if draw_radius_of_gyration:
        center_of_mass = np.sum(a,axis=0)/float(len(a))
        radius_of_gyration = np.sqrt(1/float(len(a))*np.sum(np.square(a-center_of_mass)))
        u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
        
        x = np.cos(u)*np.sin(v)*radius_of_gyration + center_of_mass[0]
        y = np.sin(u)*np.sin(v)*radius_of_gyration + center_of_mass[1]
        z = np.cos(v)*radius_of_gyration + center_of_mass[2]
        ipv.pylab.plot_wireframe(x, y, z, color="blue")
      #  ipv.pylab.plot_surface(x,y,z)
    if draw_overlaps:
        o = calculate_overlaps(a)
        if len(o)>0:
            ipv.pylab.scatter(o[:,0],o[:,1],o[:,2],color='yellow')
            
        
    ipv.pylab.plot(a[:,0],a[:,1],a[:,2],color='black')
    ipv.pylab.xyzlabel(' ',' ',' ')
    
    max = np.max(a)
    min = np.min(a)
    ipv.pylab.xlim(min, max)
    ipv.pylab.ylim(min, max)
    ipv.pylab.zlim(min, max)
    ipv.show()
    
mu = ipw.widgets.RadioButtons(
    options=['freely_jointed', 'freely_rotating'],
    description='Model:',
    disabled=False
)    
n = ipw.widgets.IntText(
    value=10,
    description='N:',
    disabled=False,
continuous_update=False)

theta = ipw.widgets.FloatSlider(
    value=90,
    min=0,
    max=180,
    step=0.1,
    
    description='theta:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

draw_end_to_end = ipw.widgets.Checkbox(
    value=False,
    description='draw end-to-end vector',
    disabled=False
)
draw_radius_of_gyration = ipw.widgets.Checkbox(
    value=False,
    description='draw radius of gyration',
    disabled=False
)

draw_overlaps = ipw.widgets.Checkbox(
    value=False,
    description='draw monomer contacts',
    disabled=False
)


ToggleButton(value=False, description='Show code')

# Chain models
## freely jointed chain and rotating chain model

In [4]:
u = ipw.interactive(f,{'manual': True},model=mu,theta=theta,n=n,draw_end_to_end=draw_end_to_end,draw_radius_of_gyration=draw_radius_of_gyration,draw_overlaps=draw_overlaps);
display(u)

interactive(children=(RadioButtons(description='Model:', options=('freely_jointed', 'freely_rotating'), value=…